In [2]:
import matplotlib.pyplot as plt 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd 
import seaborn as sns

In [5]:
import cv2

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
from tqdm import tqdm
import os
from sklearn.utils import shuffle


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from tensorflow.keras.applications import EfficientNetB0

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
import io

In [14]:
from PIL import Image

In [15]:
from IPython.display import display, clear_output

## Basic set up

In [16]:
labels = ['glioma_tumor', 'no_tumor','meningioma_tumor','pituitary_tumor']

In [17]:
X_train = []
y_train = []
os.getcwd()
os.chdir('/content/drive/MyDrive')

In [18]:
#scaling our images to 150px by 150px to make training easier/convenient
scaleDown = 150
#these 2 for loops are horrendous i know, i can't be bothered to make them more elegant, sorry
for i in labels: 
    folderPath = os.path.join('Training', i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath, j))
        img = cv2.resize(img, (scaleDown, scaleDown))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join('Testing', i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath, j))
        img = cv2.resize(img, (scaleDown, scaleDown))
        X_train.append(img)
        y_train.append(i)

X_train = np.array(X_train)
y_train = np.array(y_train)

100%|██████████| 74/74 [00:00<00:00, 114.18it/s]


In [19]:
X_train, y_train = shuffle(X_train, y_train, random_state=101)

In [20]:
X_train.shape

(3264, 150, 150, 3)

In [21]:
#The (3002, 150, 150, 3) is the final shape of our training data (the 3002 images that have been scaled to
# 150x150px with 3 channels and converted to an np array)

### Splitting the Data into Training and Testing

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.08, random_state=101)

### Converting the text based values of the y_train array into numerical values

In [23]:
y_train_final = []
for i in y_train:
    y_train_final.append(labels.index(i))
y_train = y_train_final
y_train = tf.keras.utils.to_categorical(y_train)


y_test_final = []
for i in y_test:
    y_test_final.append(labels.index(i))
y_test = y_test_final
y_test = tf.keras.utils.to_categorical(y_test)


In [24]:
effnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(scaleDown, scaleDown,3))

In [25]:
model = effnet.output

In [26]:
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4, activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [27]:
# model.summary() to get a useful summary about our model

In [28]:
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])

In [33]:
#callbacks
tensorboard = TensorBoard(log_dir = 'logs')
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='training1.chkpt',save_weights_only=True,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor=0.3, patience=2, min_delta=0.001, mode='auto', verbose=1)

In [34]:
history = model.fit(X_train,y_train,validation_split=0.08, epochs=12, verbose=1, batch_size=40, callbacks=[tensorboard, cp_callback,reduce_lr])

Epoch 1/12
70/70 [==============================] - ETA: 0s - loss: 0.1598 - accuracy: 0.9464
Epoch 1: saving model to training1.chkpt
70/70 [==============================] - 264s 4s/step - loss: 0.1598 - accuracy: 0.9464 - val_loss: 0.4087 - val_accuracy: 0.8880 - lr: 0.0010
Epoch 2/12
70/70 [==============================] - ETA: 0s - loss: 0.1027 - accuracy: 0.9660
Epoch 2: saving model to training1.chkpt
70/70 [==============================] - 257s 4s/step - loss: 0.1027 - accuracy: 0.9660 - val_loss: 0.3547 - val_accuracy: 0.9087 - lr: 0.0010
Epoch 3/12
70/70 [==============================] - ETA: 0s - loss: 0.0680 - accuracy: 0.9804
Epoch 3: saving model to training1.chkpt
70/70 [==============================] - 257s 4s/step - loss: 0.0680 - accuracy: 0.9804 - val_loss: 0.6780 - val_accuracy: 0.7925 - lr: 0.0010
Epoch 4/12
70/70 [==============================] - ETA: 0s - loss: 0.0721 - accuracy: 0.9786
Epoch 4: saving model to training1.chkpt
70/70 [========================

In [36]:
#model.save('saved_model/my_model')


TypeError: ignored

In [37]:
pred = model.predict(X_test)

9/9 [==============================] - 8s 669ms/step


In [38]:
pred = np.argmax(pred,axis=1)

In [39]:
y_test_final = np.argmax(y_test, axis=1)

In [40]:
print(classification_report(y_test_final,pred))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94        72
           1       0.98      1.00      0.99        41
           2       0.94      0.96      0.95        83
           3       0.99      1.00      0.99        66

    accuracy                           0.97       262
   macro avg       0.97      0.97      0.97       262
weighted avg       0.97      0.97      0.97       262



## Loading a new image and making a prediction using our model

In [41]:
img = Image.open(r"ImagesForPrediction/ls44dcxet1487oinyhhiqas13d5f.jpg")
opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
img = cv2.resize(opencvImage, (150,150))
img = img.reshape(1,150,150,3)
prediction = model.predict(img)
prediction = np.argmax(prediction,axis=1)[0]
    
    
if prediction==0:
    prediction='Glioma'
elif prediction==1:
    prediction='No Tumor'
elif prediction==2:
        prediction='Meningioma'
elif prediction==3:
    prediction='Pituitary Tumor'
    
print(f'The Model predicts : {prediction}')    

1/1 [==============================] - 0s 54ms/step
The Model predicts : Pituitary Tumor


In [ ]:
#@title Loading saved modules 

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

# Check its accuracy
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
